# Recommending Music to train to using an existing Spotify playlist

*Code by Sean Brockway*

### Necessary imports for Spotify and MongoDB Connections

In [1]:
import requests
import json
import pymongo
import spotipy
import spotipy.util as util 
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from spotipy.oauth2 import SpotifyClientCredentials
from pymongo import MongoClient
import urllib.parse

### Setting up connections for MongoDB and Spotify API

In [2]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

conn = MongoClient("mongodb://localhost:27017/")
db = conn['gym-music-database']
test_playlist_col = db['Murph Playlist 2']
playlist_songs = test_playlist_col.find({})

**Verifying our data is correct, this is an example of a song in our playlist**

In [3]:
print(playlist_songs[0])

{'_id': ObjectId('5eaadaaf0d1ef23f2112efcc'), 'track': 'This Is The Last Time', 'track_id': '6XW2rKHwNnX5qzk79Qis9w', 'artist_name': ['The National'], 'artist_id': ['2cCUtGK9sDU2EoElnk0GNB'], 'genres': ['chamber pop', 'indie rock', 'modern rock'], 'danceability': 0.499, 'energy': 0.496, 'key': 0, 'loudness': -10.246, 'mode': 0, 'speechiness': 0.0327, 'acousticness': 0.644, 'instrumentalness': 0.368, 'liveness': 0.0934, 'valence': 0.14, 'tempo': 140.047, 'duration_ms': 283387, 'time_signature': 4}


**Next we need to make a list of every song's genres**

In [4]:
genre_list = []
for song in playlist_songs:
    genre_list.extend(song['genres'])
genre_list

['chamber pop',
 'indie rock',
 'modern rock',
 'ambient folk',
 'art pop',
 'chamber pop',
 'chamber psych',
 'dunedin indie',
 'indie folk',
 'nz folk',
 'nz pop',
 'nz singer-songwriter',
 'ambient folk',
 'art pop',
 'chamber pop',
 'chamber psych',
 'dunedin indie',
 'indie folk',
 'nz folk',
 'nz pop',
 'nz singer-songwriter',
 'australian psych',
 'neo-psychedelic',
 'australian psych',
 'neo-psychedelic',
 'art pop',
 'chamber pop',
 'indie folk',
 'indie pop',
 'indie rock',
 'sydney indie',
 'canadian folk',
 'canadian indie',
 'canadian indie folk',
 'chamber pop',
 'deep new americana',
 'freak folk',
 'indie folk',
 'indie rock',
 'saskatchewan indie',
 'stomp and holler',
 'canadian folk',
 'canadian indie',
 'canadian indie folk',
 'chamber pop',
 'deep new americana',
 'freak folk',
 'indie folk',
 'indie rock',
 'saskatchewan indie',
 'stomp and holler',
 'canadian folk',
 'canadian indie',
 'canadian indie folk',
 'chamber pop',
 'deep new americana',
 'freak folk',
 

**Next we need to find the most common occuring genres to base our recommendations on**

In [5]:
from collections import Counter
most_common_genres = [genre for genre, genre_count in Counter(genre_list).most_common(8)]
print(most_common_genres)

['chamber pop', 'indie rock', 'indie folk', 'freak folk', 'indie pop', 'stomp and holler', 'modern rock', 'rock']


To get ideas for songs to recommend, the search end point and recommendations end point in the API can be used. For the search end point, spaces will not work, so it is best to use the "replace" method to change spaces to "%20"

The recommendations end point in the Spotify API can also be used to generate a list of song recommendations, this does not require any text to be replaced. 

In [6]:
search_genres = []
for genre in most_common_genres:
    search = genre.replace(" ", "%20")
    search_genres.append(search)
print(search_genres)

['chamber%20pop', 'indie%20rock', 'indie%20folk', 'freak%20folk', 'indie%20pop', 'stomp%20and%20holler', 'modern%20rock', 'rock']


#### Now find recommendations using the 5 most common genres. 
It is worth noting that the recommendations end point has lots of different paramaters and kwargs, and bespoke recommendations can be made using the detailed spotify audio features. For now though, we will just use genre.

Firstly we need to get all the available genre seeds used to generate recommendations

In [7]:
seeds = sp.recommendation_genre_seeds()
print(seeds['genres'])

['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-po

Now we find the genres in our playlist that can be used as a genre seed

In [12]:
list_A = most_common_genres
list_B = seeds['genres']
jVal = "|".join(list_A)

matches = [i for i in list_B if i in jVal ]
truncated_matches = matches[:-1]
print(matches)

['folk', 'indie', 'pop', 'rock']


Since there are only two matching genres and the maximum seeds are 5, we can add 3 artists as a recommendation seeds. Lets find the most common artists by firstly making a list of the artists in the playlist.

In [13]:
playlist_songs = test_playlist_col.find({})
artist_list = []
for song in playlist_songs:
    artist_list.extend(song['artist_id'])
artist_list

['2cCUtGK9sDU2EoElnk0GNB',
 '3lmR0qMiGuoIF9UC54egcG',
 '3lmR0qMiGuoIF9UC54egcG',
 '5INjqkS1o8h1imAzPqGZBb',
 '5INjqkS1o8h1imAzPqGZBb',
 '12fRkVfO2fUsz1QHgDAG3g',
 '5mFKYdmiYwNJTDtSzgFyQx',
 '5mFKYdmiYwNJTDtSzgFyQx',
 '5mFKYdmiYwNJTDtSzgFyQx',
 '5mFKYdmiYwNJTDtSzgFyQx',
 '5mFKYdmiYwNJTDtSzgFyQx',
 '2dPIBvg7mU59dCTGjhPylV',
 '5mFKYdmiYwNJTDtSzgFyQx',
 '6coAF2s4RTHN78t4UO50MM',
 '4S2f4BosAW2oF9AfGEK8ZL',
 '08GQAI4eElDnROBrJRGE0X',
 '4EVpmkEwrLYEg6jIsiPMIb',
 '4EVpmkEwrLYEg6jIsiPMIb',
 '1ThoqLcyIYvZn7iWbj8fsj',
 '3koiLjNrgRTNbOwViDipeA',
 '5mFKYdmiYwNJTDtSzgFyQx',
 '08GQAI4eElDnROBrJRGE0X',
 '3ZKyADicHqelBDeADLutiu',
 '0W79ONUwHoehEib1nRXlmi',
 '2cCUtGK9sDU2EoElnk0GNB',
 '3ZKyADicHqelBDeADLutiu',
 '5G49Sq5mMzAkGL4ZP6eVPY',
 '3ZKyADicHqelBDeADLutiu',
 '3ZKyADicHqelBDeADLutiu',
 '56ZTgzPBDge0OvCGgMO3OY',
 '2E70ENJNHoACgzTkJ3QnO4',
 '2E70ENJNHoACgzTkJ3QnO4',
 '3UvcmAOZt64oKpP95f6MMM',
 '7MhMgCo0Bl0Kukl93PZbYS',
 '2iUVQjheBnvOt8vaBrxXJz',
 '2kGBy2WHvF0VdZyqiVCkDT',
 '2kGBy2WHvF0VdZyqiVCkDT',
 

In [14]:
most_common_artists = [artist for artist, artist_count in Counter(artist_list).most_common(2)]
print(most_common_artists)

['4EVpmkEwrLYEg6jIsiPMIb', '5mFKYdmiYwNJTDtSzgFyQx']


Now we have 3 artist seeds and 2 genre seeds, we can build or query parameters

In [15]:
recommended_tracks = sp.recommendations(seed_genres = truncated_matches, seed_artists = most_common_artists, limit=20)
print(recommended_tracks['tracks'])

[{'album': {'album_type': 'ALBUM', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4EVpmkEwrLYEg6jIsiPMIb'}, 'href': 'https://api.spotify.com/v1/artists/4EVpmkEwrLYEg6jIsiPMIb', 'id': '4EVpmkEwrLYEg6jIsiPMIb', 'name': 'Fleet Foxes', 'type': 'artist', 'uri': 'spotify:artist:4EVpmkEwrLYEg6jIsiPMIb'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/62miIQWlOO88YmupzmUNGJ'}, 'href': 'https://api.spotify.com/v1/albums/62miIQWlOO88YmupzmUNGJ', 'id': '62miIQWlOO88Y

Here are the recommended tracks from Spotify, now we need to make sure there are no duplicate tracks that exist in the existing playlist, then we need to get the audio features for each track and prepare them for our classifier. 

In [16]:
for track  in recommended_tracks['tracks']:
    if any(track['id'] in songs for songs in playlist_songs):
        print('match found')
    else:
        print('no match')

no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match


No matches have been found, now we can begin getting the audio features for each song and building a data frame for our model to make predictions on. 

In [17]:
dataframe_columns = ['track', 'track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
song_list = []
for track in recommended_tracks['tracks']:
    features = sp.audio_features(track['id'])
    if (features[0] is not None):
                song_record = { 
                    "track": track['name'], 
                    "track_id": track['id'],
                    "danceability" : features[0]['danceability'],
                    "energy" : features[0]['energy'],
                    "key" : features[0]['key'],
                    "loudness" : features[0]['loudness'],
                    "mode" : features[0]['mode'],
                    "speechiness" : features[0]['speechiness'],
                    "acousticness" : features[0]['acousticness'],
                    "instrumentalness" : features[0]['instrumentalness'],
                    "liveness" : features[0]['liveness'],
                    "valence" : features[0]['valence'],
                    "tempo" : features[0]['tempo'],
                    "duration_ms" : features[0]['duration_ms'],
                    "time_signature" : features[0]['time_signature']
                } 
                song_list.append(song_record)
#print(song_list)
dfRecommendations = pd.DataFrame.from_dict(song_list)
dfRecommendations

,track,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,In the Hot Hot Rays,5V5QkXN5ks3LrTOqsWzFLD,0.553,0.6990,5,-4.938,1,0.0323,0.0461,0.007280,0.1410,0.4430,127.107,186267,4
1,Parade,2kkX9IzielZhUBybezdE0m,0.499,0.5160,9,-10.846,1,0.0280,0.1850,0.021500,0.1100,0.4850,149.886,325427,4
2,To Leave,57xpT2y8ZCBxdg9bUrl8gQ,0.621,0.0597,6,-19.450,1,0.0569,0.8400,0.000008,0.2260,0.1970,57.096,230950,4
3,no tears left to cry,2qT1uLXPVPzGgFOx4jtEuo,0.699,0.7130,9,-5.507,0,0.0594,0.0400,0.000003,0.2940,0.3540,121.993,205920,4
4,Married to the Movies,09s516xPkw50SsnjdikxLC,0.421,0.4390,2,-8.884,1,0.0344,0.6410,0.008590,0.1120,0.3920,137.499,152680,4
5,My Mother & I,2KYaAF4k5YOvbYmlyHazfO,0.425,0.2430,0,-15.128,1,0.0329,0.8780,0.024200,0.0864,0.2340,160.003,279032,3
6,Rush,6ysK6T8Gvtx1f7VBzaTSxA,0.545,0.6850,2,-5.830,1,0.0266,0.0968,0.005460,0.5390,0.5920,140.050,219627,4
7,No Vacancy,4QeoDcR16IHpmmgFGQDrCp,0.720,0.7270,5,-3.882,1,0.0489,0.0556,0.000000,0.1380,0.4810,99.994,223190,4
8,Where Are You Now (Bonus Track),3dQfMd0TxVHRnFeaudSmSL,0.493,0.3460,4,-10.147,1,0.0411,0.6270,0.000003,0.0917,0.1290,83.113,219600,4
9,Lay Down In The Tall Grass,4UdbPAvJmIJJe9KRi5giKr,0.741,0.3190,9,-11.177,1,0.0783,0.6930,0.094300,0.1430,0.4070,148.981,338373,3


Now we have a data frame which we can use to predict if a song is gym suitable or not. Before this happens, we need to train our model. We will do this with 10 thousands records of training data using the random forest algorthim. 

In [18]:
training_col = db['medium_test_songlist']
training_songs = training_col.find({})
dfTraining =  pd.DataFrame(list(training_songs))
dfTraining

,_id,track,track_id,artist_name,artist_id,from_playlist,genres,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,workout_suitable
0,5e7e4fa0b96e22bd0ee61d99,'Till I Collapse,4xkOaSrkexMciUUogZKVTS,"[Eminem, Nate Dogg]","[7dGJo4pcD2V6oG8kP0tJRR, 1Oa0bMld0A3u5OTYfMzp5h]","Gym Motivation : Workout Motivation, Training ...","[detroit hip hop, g funk, hip hop, rap, g funk...",0.548,0.8470,1,...,1,0.1860,0.06220,0.000000,0.0816,0.1000,171.447,297787,4,1
1,5e7e4fa1b96e22bd0ee61d9a,Numb / Encore,5sNESr6pQfIhL3krM8CtZn,"[JAY-Z, Linkin Park]","[3nFkdlSjzX9mRTtwJOzDYB, 6XyY86QOPPrYVGvF9ch6wz]","Gym Motivation : Workout Motivation, Training ...","[east coast hip hop, hip hop, pop rap, rap, al...",0.687,0.7930,2,...,1,0.1660,0.06030,0.000000,0.5820,0.7510,107.045,205733,4,1
2,5e7e4fa1b96e22bd0ee61d9b,Remember the Name (feat. Styles of Beyond),6ndmKwWqMozN2tcZqzCX4K,"[Fort Minor, Styles Of Beyond]","[7dWYWUbO68rXJOcyA7SpJk, 5bf6yYgHODBW5EreBZshpX]","Gym Motivation : Workout Motivation, Training ...","[rap rock, cali rap, rap rock]",0.688,0.8350,8,...,1,0.0911,0.05830,0.000003,0.0795,0.8800,84.858,230493,4,1
3,5e7e4fa1b96e22bd0ee61d9c,In Da Club,4RY96Asd9IefaL3X4LOLZ8,[50 Cent],[3q7HBObVc0L8jNeTe5Gofh],"Gym Motivation : Workout Motivation, Training ...","[east coast hip hop, gangster rap, hip hop, po...",0.902,0.7200,6,...,0,0.3470,0.26000,0.000000,0.0749,0.8050,90.059,193467,4,1
4,5e7e4fa1b96e22bd0ee61d9d,Stronger,4fzsfWzRhPawzqhX8Qt9F3,[Kanye West],[5K4W6rqBFWDnAN6FQUkS6x],"Gym Motivation : Workout Motivation, Training ...","[chicago rap, rap]",0.617,0.7170,10,...,0,0.1530,0.00564,0.000000,0.4080,0.4900,103.992,311867,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12678,5e7e63f38aa9abc22c18141a,Hallelujah,0uCGsNZqjHgiYO4BwN6Cjw,[Ben Laver],[3lTGvG2QAIoGMp7BKeH4C0],Calming Instrumental Covers,[],0.428,0.0415,7,...,0,0.0476,0.98500,0.942000,0.1060,0.2810,71.353,165882,4,0
12679,5e7e63f48aa9abc22c18141b,Close To Me,0kPLGFA5BjDGVkDbu4bLzE,[Mia Carrera],[00zoOzjXViIF4Fs8XQSK9m],Calming Instrumental Covers,"[calming instrumental, piano cover]",0.636,0.2410,4,...,1,0.0494,0.99000,0.902000,0.1510,0.1210,102.094,203587,4,0
12680,5e7e63f48aa9abc22c18141c,Love Me Like You Do,1nyxHNRjeyTlJQwsLMH7C0,[Anna Kitkowska],[45Miu8OyhYvkkQBXhn0MfQ],Calming Instrumental Covers,"[calming instrumental, chill guitar]",0.635,0.2310,11,...,0,0.0419,0.93900,0.947000,0.0925,0.3210,144.229,198208,4,0
12681,5e7e63f48aa9abc22c18141d,Fix You,2ZzqC4NSi374IufPYvGMP6,[Music Lab Collective],[1ylcY77FWeSVQKh5et1VGp],Calming Instrumental Covers,[classify],0.667,0.0498,3,...,1,0.0673,0.99100,0.890000,0.0818,0.0436,125.070,332427,4,0


Here is the dataframe we will use to train our classifier, next we define the features we want the classifier to learn and the feature we want to predict. These are named X and y respectively.

In [19]:
feature_cols = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'mode', 'speechiness', 'tempo', 'time_signature']
X = dfTraining.loc[:, feature_cols]
y = dfTraining.workout_suitable

In [20]:
X

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,mode,speechiness,tempo,time_signature
0,0.06220,0.548,297787,0.8470,0.000000,1,0.0816,1,0.1860,171.447,4
1,0.06030,0.687,205733,0.7930,0.000000,2,0.5820,1,0.1660,107.045,4
2,0.05830,0.688,230493,0.8350,0.000003,8,0.0795,1,0.0911,84.858,4
3,0.26000,0.902,193467,0.7200,0.000000,6,0.0749,0,0.3470,90.059,4
4,0.00564,0.617,311867,0.7170,0.000000,10,0.4080,0,0.1530,103.992,4
...,...,...,...,...,...,...,...,...,...,...,...
12678,0.98500,0.428,165882,0.0415,0.942000,7,0.1060,0,0.0476,71.353,4
12679,0.99000,0.636,203587,0.2410,0.902000,4,0.1510,1,0.0494,102.094,4
12680,0.93900,0.635,198208,0.2310,0.947000,11,0.0925,0,0.0419,144.229,4
12681,0.99100,0.667,332427,0.0498,0.890000,3,0.0818,1,0.0673,125.070,4


In [21]:
y

0        1
1        1
2        1
3        1
4        1
        ..
12678    0
12679    0
12680    0
12681    0
12682    0
Name: workout_suitable, Length: 12683, dtype: int64

## Prediction

#### Constructing a prediction model using X and y

In [22]:
rf = RandomForestClassifier(n_estimators=40)
workout_classifier = rf.fit(X, y)
workout_classifier

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Now we prepare the song recommendations to be tested 

In [23]:
X_predict = dfRecommendations.loc[:, feature_cols]
X_predict.shape

(20, 11)

### Using the classifer to predict the recommendations (X_predict)

In [24]:
new_prediction_class = workout_classifier.predict(X_predict)
new_prediction_class

array([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1])

 This is the array of predictions, we can build a new dataframe to display this information more clearly

In [25]:
dfClassifiedRecommendations = pd.DataFrame({
    'Track Name': dfRecommendations.track,
    'Track_id': dfRecommendations.track_id,
    'Prediction': new_prediction_class
})
dfClassifiedRecommendations

,Track Name,Track_id,Prediction
0,In the Hot Hot Rays,5V5QkXN5ks3LrTOqsWzFLD,1
1,Parade,2kkX9IzielZhUBybezdE0m,0
2,To Leave,57xpT2y8ZCBxdg9bUrl8gQ,0
3,no tears left to cry,2qT1uLXPVPzGgFOx4jtEuo,1
4,Married to the Movies,09s516xPkw50SsnjdikxLC,0
5,My Mother & I,2KYaAF4k5YOvbYmlyHazfO,0
6,Rush,6ysK6T8Gvtx1f7VBzaTSxA,0
7,No Vacancy,4QeoDcR16IHpmmgFGQDrCp,1
8,Where Are You Now (Bonus Track),3dQfMd0TxVHRnFeaudSmSL,0
9,Lay Down In The Tall Grass,4UdbPAvJmIJJe9KRi5giKr,0


Here we have a list of songs with the classifiers prediction of if it is suitable for working out to or not. 
Now all that is left to do is display only the songs predicted as suitable for our hypothetical user.

In [26]:
dict1 = dfClassifiedRecommendations.to_dict('records')
for track in dict1:
    print(track)

{'Track Name': 'In the Hot Hot Rays', 'Track_id': '5V5QkXN5ks3LrTOqsWzFLD', 'Prediction': 1}
{'Track Name': 'Parade', 'Track_id': '2kkX9IzielZhUBybezdE0m', 'Prediction': 0}
{'Track Name': 'To Leave', 'Track_id': '57xpT2y8ZCBxdg9bUrl8gQ', 'Prediction': 0}
{'Track Name': 'no tears left to cry', 'Track_id': '2qT1uLXPVPzGgFOx4jtEuo', 'Prediction': 1}
{'Track Name': 'Married to the Movies', 'Track_id': '09s516xPkw50SsnjdikxLC', 'Prediction': 0}
{'Track Name': 'My Mother & I', 'Track_id': '2KYaAF4k5YOvbYmlyHazfO', 'Prediction': 0}
{'Track Name': 'Rush', 'Track_id': '6ysK6T8Gvtx1f7VBzaTSxA', 'Prediction': 0}
{'Track Name': 'No Vacancy', 'Track_id': '4QeoDcR16IHpmmgFGQDrCp', 'Prediction': 1}
{'Track Name': 'Where Are You Now (Bonus Track)', 'Track_id': '3dQfMd0TxVHRnFeaudSmSL', 'Prediction': 0}
{'Track Name': 'Lay Down In The Tall Grass', 'Track_id': '4UdbPAvJmIJJe9KRi5giKr', 'Prediction': 0}
{'Track Name': 'Covered in Dust', 'Track_id': '0c4VvfbtuVs5GVGYC9SdG9', 'Prediction': 0}
{'Track Name'

In [27]:
dictRecommend = []
for track in dict1: 
    if(track['Prediction'] == 1):
        dictRecommend.append(track)
for track in dictRecommend:
    print(track)

{'Track Name': 'In the Hot Hot Rays', 'Track_id': '5V5QkXN5ks3LrTOqsWzFLD', 'Prediction': 1}
{'Track Name': 'no tears left to cry', 'Track_id': '2qT1uLXPVPzGgFOx4jtEuo', 'Prediction': 1}
{'Track Name': 'No Vacancy', 'Track_id': '4QeoDcR16IHpmmgFGQDrCp', 'Prediction': 1}
{'Track Name': 'White Winter Hymnal', 'Track_id': '3QVtICc8ViNOy4I5K14d8Z', 'Prediction': 1}
{'Track Name': 'Operating', 'Track_id': '2cq3mFSgiXna6SnUH9eIts', 'Prediction': 1}


## The recommendations are:

In [28]:
dfResults = pd.DataFrame.from_dict(dictRecommend)
dfResults

,Track Name,Track_id,Prediction
0,In the Hot Hot Rays,5V5QkXN5ks3LrTOqsWzFLD,1
1,no tears left to cry,2qT1uLXPVPzGgFOx4jtEuo,1
2,No Vacancy,4QeoDcR16IHpmmgFGQDrCp,1
3,White Winter Hymnal,3QVtICc8ViNOy4I5K14d8Z,1
4,Operating,2cq3mFSgiXna6SnUH9eIts,1
